In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.stats import zscore
from scipy import stats
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/stock-market-dataset/Train_full.csv
/kaggle/input/stock-market-dataset/Description.docx
/kaggle/input/stock-market-dataset/Test_small_features.csv


Đọc dữ liệu

In [2]:
data = pd.read_csv('/kaggle/input/stock-market-dataset/Train_full.csv')
data = data.drop(columns = data.columns[0])

**Xem tất cả các đặc trưng của dữ liệu**

In [3]:
data.columns.values

array(['Open', 'High', 'Low', 'Close', 'Volume', 'body', 'upper_tail',
       'lower_tail', 'SMA_50', 'SMA_20', 'ATR', 'CCI', 'SAR', 'hour',
       'min', 'dayofweek', 'JPY', 'AUD', 'EUR', 'GBP', 'USD',
       'lag_return_1', 'return_2', 'lag_return_2', 'return_3',
       'lag_return_3', 'return_4', 'lag_return_4', 'return_5',
       'lag_return_5', 'return_6', 'lag_return_6', 'return_7',
       'lag_return_7', 'return_8', 'lag_return_8', 'return_9',
       'lag_return_9', 'return_10', 'lag_return_10', 'return_11',
       'lag_return_11', 'return_12', 'lag_return_12', 'return_13',
       'lag_return_13', 'return_14', 'lag_return_14', 'return_15',
       'lag_return_15', 'return_16', 'lag_return_16', 'return_17',
       'lag_return_17', 'return_18', 'lag_return_18', 'return_19',
       'lag_return_19', 'return_20', 'lag_return_20', 'return_21',
       'lag_return_21', 'return_22', 'lag_return_22', 'return_23',
       'lag_return_23', 'return_24', 'lag_return_24', 'return_25',
       'la

Nhìn vào dữ liệu ta thấy thiếu đặc trưng return_1, theo tính chất của dữ liệu thì return_1 = return_2 - lag_return_1


In [4]:
data['return_1'] = data['return_2'] - data['lag_return_1']
data.shape

(709314, 214)

**Kiểm tra giá trị còn thiếu trong dữ liệu**

In [5]:
data.isnull().values.sum()

0

**Lọc ngoại lệ với phương Z_Score với ngưỡng =  4 **

In [6]:
z_scores = stats.zscore(data)
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 4).all(axis=1)
df = data[filtered_entries]

In [7]:
df.shape

(501472, 214)

Chia ra biến đoc lập và biến phụ thuộc

In [8]:
Y = df["up_down"] #independent columns
X = df.drop(columns="up_down")
columns = X.columns


Sử dụng MinMax để chuấn hóa dữ liệu

In [9]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_scale = scaler.fit_transform(X)

Các features_importances đã được chọn cho vào một list

In [10]:
feature_set = ['Close', 'High', 'Low', 'return_1', 'return_2', 'return_3']


In [11]:
x=X[feature_set]
y=Y
#x = preprocessing.scale(x) 

Sử dụng Kfold để chia dữ liệu test

In [12]:
##from sklearn.model_selection import train_test_split
##X_train,X_test,Y_train,Y_test = train_test_split(x, y, test_size = 0.2, random_state = 0)
kf = KFold(n_splits=10)
kf.get_n_splits(x)
print(kf)
KFold(n_splits=10, random_state=None, shuffle=False)
i=0
A=0
P=0
R=0
F=0
for train_index, test_index in kf.split(x):
    X_train, X_test = x.iloc[train_index], x.iloc[test_index]     
    Y_train, Y_test = Y.iloc[train_index], Y.iloc[test_index]
    i=i+1
    print("So luong mau tap train ",i,": ",X_train.shape[0])
    print("So luong mau tap test ",i,": ",X_test.shape[0])
    import lightgbm as lgb
    model = lgb.LGBMClassifier()
    model.fit(X_train, Y_train)
    Y_pred = model.predict(X_test)
    accuracy = accuracy_score(Y_test,Y_pred)
    print("Accuracy = ",(accuracy))
    A=A+accuracy
    from sklearn.metrics import precision_score
    print("Precision = ",precision_score(Y_test, Y_pred))
    P=P+precision_score(Y_test, Y_pred)
    from sklearn.metrics import recall_score
    print("Recall = ",recall_score(Y_test, Y_pred))
    R=R+recall_score(Y_test, Y_pred)
    from sklearn.metrics import f1_score
    print("F-measure = ",f1_score(Y_test, Y_pred))
    F=F+f1_score(Y_test, Y_pred)
print("Trung binh Accuracy = ", A/10)
print("Trung binh Precision = ", P/10)
print("Trung binh Recall = ", R/10)
print("Trung binh F-measure = ",F/10)

KFold(n_splits=10, random_state=None, shuffle=False)
Accuracy =  0.7123115577889447
Precision =  0.5268156424581005
Recall =  0.4371812702828002
F-measure =  0.4778312642513301
Accuracy =  0.7924343941931882
Precision =  0.5238147984049624
Recall =  0.4362948611495525
F-measure =  0.476065837821513
Accuracy =  0.7059445231020799
Precision =  0.5309623739047198
Recall =  0.4626291140052608
F-measure =  0.4944459681843116
Accuracy =  0.7380900153548567
Precision =  0.5229738072140401
Recall =  0.43427316001464666
F-measure =  0.47451388333199973
Accuracy =  0.7379105430035695
Precision =  0.5223248326696602
Recall =  0.4509875640087784
F-measure =  0.4840419267459663
Accuracy =  0.7009192972660379
Precision =  0.5237138263665595
Recall =  0.41794970490120603
F-measure =  0.4648922506065364
Accuracy =  0.692424272638443
Precision =  0.5325895007224274
Recall =  0.408659768415866
F-measure =  0.4624660207708928
Accuracy =  0.7025345484276228
Precision =  0.5262863087140914
Recall =  0.4223